The Lake Model as preppared by the EMA Workbench:
Kwakkel, Jan H. "The Exploratory Modeling Workbench: An open source toolkit for exploratory modeling, scenario discovery, and (multi-objective) robust decision making." Environmental Modelling & Software 96 (2017): 239-250.

Peterson, G.D., Carpenter, S.R. and Brock, W.A. (2003), UNCERTAINTY AND THE MANAGEMENT OF MULTISTATE ECOSYSTEMS: AN APPARENTLY RATIONAL ROUTE TO COLLAPSE. Ecology, 84: 1403-1411. https://doi.org/10.1890/0012-9658(2003)084[1403:UATMOM]2.0.CO;2

In [49]:

import random as rand
from scipy.optimize import brentq as root
import math
import numpy as np

In [50]:



# Construct the lake problem
def lake_problem(pollution_limit_low,
                 pollution_limit_high,
         b = 0.42,        # decay rate for P in lake (0.42 = irreversible)
         q = 2.0,         # recycling exponent
         mean = 0.02,     # mean of natural inflows
         stdev = 0.001,   # standard deviation of natural inflows
         alpha = 0.4,     # utility from pollution
         delta = 0.98,    # future utility discount rate
         myears = 150,
         nsamples = 100): # monte carlo sampling of natural inflows
    pollution_limit = [(rand.random()*pollution_limit_high)+pollution_limit_low  for i in  range(myears)]
    Pcrit = root(lambda x: x**q/(1+x**q) - b*x, 0.01, 1.5)
    nvars = len(pollution_limit)
    X = np.zeros((nvars,))
    average_daily_P = np.zeros((nvars,))
    print(pollution_limit)
    decisions = np.array(pollution_limit)
    reliability = 0.0

    for _ in range(nsamples):
        X[0] = 0.0

        natural_inflows = np.random.lognormal(
                math.log(mean**2 / math.sqrt(stdev**2 + mean**2)),
                math.sqrt(math.log(1.0 + stdev**2 / mean**2)),
                size = nvars)

        for t in range(1,nvars):
            X[t] = (1-b)*X[t-1] + X[t-1]**q/(1+X[t-1]**q) + decisions[t-1] + natural_inflows[t-1]
            average_daily_P[t] += X[t]/float(nsamples)

        reliability += np.sum(X < Pcrit)/float(nsamples*nvars)

    max_P = np.max(average_daily_P)
    utility = np.sum(alpha*decisions*np.power(delta,np.arange(nvars)))
    inertia = np.sum(np.diff(decisions) > -0.02)/float(nvars-1)

    return (max_P, utility, inertia, reliability)

In [51]:
b : 'CWLFloatInput' = 0.33
q : 'CWLFloatInput' = 2
mean : 'CWLFloatInput' = 0.05
stdev : 'CWLFloatInput' = 0.004
delta : 'CWLFloatInput' = 0.99
alpha : 'CWLFloatInput' = 0.41
nsamples : 'CWLIntInput' = 150
myears : 'CWLIntInput' = 100
pollution_limit_low : 'CWLFloatInput'
pollution_limit_high : 'CWLFloatInput'

output_file : 'CWLFilePathOutput' = "./y.csv"

In [52]:
y = lake_problem(pollution_limit_low=pollution_limit_low,
    pollution_limit_high=pollution_limit_high,
    b= b,
    q=q,
    mean=mean,
    stdev=stdev,
    delta=delta,
    alpha=alpha,
    myears=myears,
    nsamples=nsamples,

   )

[0.08439728412749858, 0.0992144485077274, 0.09354866429236922, 0.08363531173366975, 0.0174797630334963, 0.030568996846305953, 0.052394676939472896, 0.06214731032413334, 0.0837205089105773, 0.05764784294044333, 0.0473377314841704, 0.06600766658283312, 0.07481942237029758, 0.04443627838497286, 0.032796125005418744, 0.08237369075364848, 0.07833078883009725, 0.013092178978184677, 0.01075721435967101, 0.02734355197318742, 0.07692309799366086, 0.06950999884567831, 0.05686098947851265, 0.02479808626859589, 0.03063540610039586, 0.09472886438563977, 0.07243797757643079, 0.0654318383819108, 0.0006407745121528974, 0.004725985084814955, 0.08191434404088367, 0.05857512193511591, 0.08649163518450964, 0.012400677834266939, 0.02220184017014948, 0.0646282739595629, 0.020603837374080114, 0.06047637854385595, 0.08669014607158256, 0.01819336031881377, 0.017255338853050962, 0.07280012306894028, 0.07783973956443846, 0.05193335905952794, 0.03751224113433815, 0.09517471151009416, 0.08007421933157394, 0.035421

In [53]:
np.savetxt(output_file,y)